In [1]:
from load_api_key import load_openai_api_key
key = load_openai_api_key()
from prompt_helpers import get_prompt, generate_code, run_llm_code
from interpret.glassbox import ExplainableBoostingClassifier, ExplainableBoostingRegressor
import joblib
import t2ebm.graphs as graphs
from openai import OpenAI



ebm = joblib.load("../trained_ebm.pkl")
graph = graphs.extract_graph(ebm, 0)

API Key loaded successfully.


In [2]:
prompt = get_prompt(graph)
print(prompt)


    The Explainable Boosting Machine (EBM) shape function graph for a specific feature is loaded as the `EBMGraph` object.

    Feature analyzed: "Cigarettes per Day" (Type: continuous).

    x-values and corresponding shape scores represent the relationship between this feature and the target:
    - X-values (ranges or categories): [(0.0, 0.5), (0.5, 1.5), (1.5, 2.5), (2.5, 3.5), (3.5, 4.5), (4.5, 5.5), (5.5, 6.5), (6.5, 7.5), (7.5, 8.5), (8.5, 9.5), (9.5, 10.5), (10.5, 11.5), (11.5, 12.5), (12.5, 13.5), (13.5, 14.5), (14.5, 15.5), (15.5, 16.5), (16.5, 17.5), (17.5, 18.5), (18.5, 19.5), (19.5, 20.0)]
    - Shape function scores: [-9.79172542 -8.78941533 -7.79184658 -6.78866511 -5.79283312 -4.79011128
 -3.79072316 -2.79014776 -1.79144518 -0.79372247  0.21082704  1.20891648
  2.20839762  3.21115237  4.20799474  5.20810371  6.20715089  7.21100573
  8.20904816  9.20798923 10.20866302]
    - Standard deviations (uncertainty at each x-value): [0.00326436 0.00400482 0.00550522 0.00585849 0.

In [3]:
messages = [
    {
        "role": "system",
        "content": "You are an expert datascientist assistant solving Kaggle problems. You answer only by generating code. Answer as concisely as possible.",
    },
    {
        "role": "user",
        "content": prompt,
    },
]
client = OpenAI(api_key=key)

code = generate_code(messages, client)
code

"\n# The identified anomaly is a sudden score jump from 9.20798923 to 10.20866302 between x-values (19.5, 20.0) and the previous score (9.20798923).\n# Correction rationale: This jump is inconsistent with the expected smooth progression of scores for a continuous feature, suggesting a lack of continuity in the model's output.\n# Corrective code:\nfor i, (x_val, score) in enumerate(zip([(0.0, 0.5), (0.5, 1.5), (1.5, 2.5), (2.5, 3.5), (3.5, 4.5), (4.5, 5.5), (5.5, 6.5), (6.5, 7.5), (7.5, 8.5), (8.5, 9.5), (9.5, 10.5), (10.5, 11.5), (11.5, 12.5), (12.5, 13.5), (13.5, 14.5), (14.5, 15.5), (15.5, 16.5), (16.5, 17.5), (17.5, 18.5), (18.5, 19.5), (19.5, 20.0)], [-9.79172542, -8.78941533, -7.79184658, -6.78866511, -5.79283312, -4.79011128, -3.79072316, -2.79014776, -1.79144518, -0.79372247, 0.21082704, 1.20891648, 2.20839762, 3.21115237, 4.20799474, 5.20810371, 6.20715089, 7.21100573, 8.20904816, 9.20798923, 10.20866302])):\n    if i == 19:  # Condition for the identified anomaly\n        ebm_

In [4]:
print(code)


# The identified anomaly is a sudden score jump from 9.20798923 to 10.20866302 between x-values (19.5, 20.0) and the previous score (9.20798923).
# Correction rationale: This jump is inconsistent with the expected smooth progression of scores for a continuous feature, suggesting a lack of continuity in the model's output.
# Corrective code:
for i, (x_val, score) in enumerate(zip([(0.0, 0.5), (0.5, 1.5), (1.5, 2.5), (2.5, 3.5), (3.5, 4.5), (4.5, 5.5), (5.5, 6.5), (6.5, 7.5), (7.5, 8.5), (8.5, 9.5), (9.5, 10.5), (10.5, 11.5), (11.5, 12.5), (12.5, 13.5), (13.5, 14.5), (14.5, 15.5), (15.5, 16.5), (16.5, 17.5), (17.5, 18.5), (18.5, 19.5), (19.5, 20.0)], [-9.79172542, -8.78941533, -7.79184658, -6.78866511, -5.79283312, -4.79011128, -3.79072316, -2.79014776, -1.79144518, -0.79372247, 0.21082704, 1.20891648, 2.20839762, 3.21115237, 4.20799474, 5.20810371, 6.20715089, 7.21100573, 8.20904816, 9.20798923, 10.20866302])):
    if i == 19:  # Condition for the identified anomaly
        ebm_graph.s

In [8]:
new_graph = run_llm_code(code, graph)

In [14]:
graph.scores[-2]

9.207989232671853

In [15]:
new_graph.scores[-2]

9.70798923